# BCG Argo data

In [1]:
# Loading libraries
import argopy as ar
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogLocator

In [3]:
ds = ar.DataFetcher().float(3901586).to_xarray()

ValueError: zero-size array to reduction operation maximum which has no identity